## STEP 1 : LOAD THE DATASET

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')

### Step 1a : Visualilzing the dataset

In [ ]:
df.head()

### Step 1b : Preprocessing the data
This step is important to scale our data to nominal values for Classification problem, We only scale data and not the targets. Separating the data (image pixels) from the target values is also done

In [ ]:
targets = pd.get_dummies(df.pop('label')).values
data = df.values

In [ ]:
targets.shape

In [ ]:
data.shape

In [ ]:
input_shape = (28,28, 1) # 28*28 = 784

In [ ]:
from sklearn.preprocessing import minmax_scale
data = minmax_scale(data)

### Reshape data

In [ ]:
data = np.reshape(data,(-1, 28, 28,1))

In [ ]:
data.shape

## STEP 2 : SPLITTING THE DATA INTO TRAIN AND TEST SET 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, targets , test_size = 0.2, random_state=0)

## STEP 3:  APPLYING THE REGRESSOR

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(targets.shape[1],activation=tf.nn.softmax))

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=20)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

## STEP 4:  EVALUATE MODEL PERFORMANCE

In [ ]:
print(f"Accuracy : {accuracy}")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.argmax(y_pred, axis =1)
y_test = np.argmax(y_test, axis =1)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
print("Accuracy :" + str(round(accuracy_score(y_test, y_pred),4)))
p, r, f, s = precision_recall_fscore_support(y_test, y_pred, average='macro')
print(round(p,4),round(r,4),round(f,4))